In [61]:
import os
import pandas as pd
import numpy as np
from scipy.stats import mode
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from xgboost import XGBClassifier

SEED = 42 # Muito importante manter a SEED igual em todos os modelos para garantir a consistência dos dados no ensemble
FOLDS = 5 # Muito importante manter o mesmo número de FOLDS em todos os modelos para garantir a consistência dos dados no ensemble

In [62]:
sintetico = pd.read_csv('../../src/train/train.csv', index_col='id')
original = pd.read_csv('../../src/train/original.csv')
test = pd.read_csv('../../src/test/test.csv', index_col='id')

train = pd.concat([sintetico, original], ignore_index=True)

initial_features = list(test.columns)

In [63]:
def cross_validation(model, X, y, encoder, scoring=accuracy_score):

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

    scores = []
    out_of_fold = []
    
    for i, (train_index, val_index) in enumerate(skf.split(X, y)):
        print(f"Fold {i + 1}")
        
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]

        X_val = X.iloc[val_index]
        y_val = y.iloc[val_index]
        
        model.fit(X_train, y_train)

        probabilidades = model.predict_proba(X_val)

        # Recuperar a predição final a partir das probabilidades
        indices_predicoes = np.argmax(probabilidades, axis=1)
        classes_preditas = model.classes_[indices_predicoes]

        score = scoring(y_val, classes_preditas)

        scores.append(score)

        true_label = pd.Series(y_val.values, name='true')

        pred_label_df = pd.DataFrame(probabilidades)

        oof_pred = pd.concat([pred_label_df, true_label], axis=1, ignore_index=True)
        oof_pred.columns = [f'pred_{encoder.classes_[model.classes_[0]]}', f'pred_{encoder.classes_[model.classes_[1]]}', f'pred_{encoder.classes_[model.classes_[2]]}', 'true']

        out_of_fold.append(oof_pred)

    print(f"Score: {np.mean(scores)}")
    
    return scores, out_of_fold

In [64]:
def save_oof(oof):

    os.makedirs('oof', exist_ok=True)

    for i, fold in enumerate(oof):
        fold.to_csv(f'oof/fold_{i+1}.csv', index=False)

In [65]:
def predict_test(model, X_train, y_train, X_test, encoder):

    model.fit(X_train, y_train)

    probabilidades = model.predict_proba(X_test)
    pred_label_df = pd.DataFrame(probabilidades)

    pred_label_df.columns = [f'pred_{encoder[model.classes_[0]]}', f'pred_{encoder[model.classes_[1]]}', f'pred_{encoder[model.classes_[2]]}']

    os.makedirs('test', exist_ok=True)

    pred_label_df.to_csv(f'test/test_pred.csv', index=False)

    return pred_label_df

In [66]:
def PreprocessDatapreprocess_data(df):
    df = df.copy()
    df['total'] = df.sum(axis = 1)
    df['mean'] = df.mean(axis = 1)
    df['median'] = df.median(axis = 1)
    df['std'] = df.std(axis = 1)
    df['min'] = df.min(axis = 1)
    df['max'] = df.max(axis = 1)
    df['ptp'] = df.values.ptp(axis = 1)
    df['q25'] = df.quantile(0.25, axis = 1)
    df['q75'] = df.quantile(0.75, axis = 1)
    df['credited_avg'] = (df['Curricular units 1st sem (credited)'] + df['Curricular units 2nd sem (credited)']) / 2
    df['enrolled_avg'] = (df['Curricular units 1st sem (enrolled)'] + df['Curricular units 2nd sem (enrolled)']) / 2
    df['evaluations_avg'] = (df['Curricular units 1st sem (evaluations)'] + df['Curricular units 2nd sem (evaluations)']) / 2
    df['approved_avg'] = (df['Curricular units 1st sem (approved)'] + df['Curricular units 2nd sem (approved)']) / 2
    df['grade_avg'] = (df['Curricular units 1st sem (grade)'] + df['Curricular units 2nd sem (grade)']) / 2
    df['wo_evaluations_avg'] = (df['Curricular units 1st sem (without evaluations)'] + 
                                df['Curricular units 2nd sem (without evaluations)']) / 2
    df['Age_at_enrollment_cat'] = pd.cut(df['Age at enrollment'], 
                                            bins = [18, 24, 30, 35, 40], 
                                            labels = [1, 2, 3, 4])
    df['Parents_qualification_avg'] = (df["Mother's qualification"] + df["Father's qualification"]) / 2
    df['Parents_occupation_avg'] = (df["Mother's occupation"] + df["Father's occupation"]) / 2
    df['Unemployment_Inflation_ratio'] = (df['Unemployment rate'] + 1) / (df['Inflation rate'] + 1)
    
    return df

In [67]:
from sklearn.base import BaseEstimator, TransformerMixin

class PreprocessData(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = X.copy()
        df['total'] = df.sum(axis=1)
        df['mean'] = df.mean(axis=1)
        df['median'] = df.median(axis=1)
        df['std'] = df.std(axis=1)
        df['min'] = df.min(axis=1)
        df['max'] = df.max(axis=1)
        df['ptp'] = df.values.ptp(axis=1)
        df['q25'] = df.quantile(0.25, axis=1)
        df['q75'] = df.quantile(0.75, axis=1)
        df['credited_avg'] = (df['Curricular units 1st sem (credited)'] + df['Curricular units 2nd sem (credited)']) / 2
        df['enrolled_avg'] = (df['Curricular units 1st sem (enrolled)'] + df['Curricular units 2nd sem (enrolled)']) / 2
        df['evaluations_avg'] = (df['Curricular units 1st sem (evaluations)'] + df['Curricular units 2nd sem (evaluations)']) / 2
        df['approved_avg'] = (df['Curricular units 1st sem (approved)'] + df['Curricular units 2nd sem (approved)']) / 2
        df['grade_avg'] = (df['Curricular units 1st sem (grade)'] + df['Curricular units 2nd sem (grade)']) / 2
        df['wo_evaluations_avg'] = (df['Curricular units 1st sem (without evaluations)'] + 
                                    df['Curricular units 2nd sem (without evaluations)']) / 2
        df['Age_at_enrollment_cat'] = pd.cut(df['Age at enrollment'], 
                                                bins=[18, 24, 30, 35, 40], 
                                                labels=[1, 2, 3, 4])
        df['Parents_qualification_avg'] = (df["Mother's qualification"] + df["Father's qualification"]) / 2
        df['Parents_occupation_avg'] = (df["Mother's occupation"] + df["Father's occupation"]) / 2
        df['Unemployment_Inflation_ratio'] = (df['Unemployment rate'] + 1) / (df['Inflation rate'] + 1)
        
        return df


In [68]:
le = LabelEncoder()

y = pd.DataFrame(le.fit_transform(train['Target']), columns=['Target'])
y = y['Target']
X = train[initial_features]

In [69]:
xgb_best_params =  {'n_estimators': 1168, 
                   'learning_rate': 0.021899695434225392, 
                   'max_depth': 9, 
                   'min_child_weight': 7, 
                   'subsample': 0.8833415189829511, 
                   'colsample_bytree': 0.47870555779903173, 
                   'reg_alpha': 0.003962141750526502, 
                   'reg_lambda': 0.0005483724027167153, 
                   'gamma': 0.9706261279770567, 
                   'scale_pos_weight': 29.598024870063032}

# best_catb_params = {'iterations': 1060, 
#                     'learning_rate': 0.12870938526369563, 
#                     'depth': 5, 
#                     'min_data_in_leaf': 2, 
#                     'colsample_bylevel': 0.5571138654353053, 
#                     'l2_leaf_reg': 3.917650301267089, 
#                     'random_strength': 4.0271592863936245, 
#                     'bagging_temperature': 0.20975982889738484}

In [70]:
# Pipeline de pré-processamento + escalonamento
preprocess_and_scale_pipeline = Pipeline([
    ('fet_eng', PreprocessData()),
    ('scaler', MinMaxScaler())
])

# ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('fet_eng_and_scaling', preprocess_and_scale_pipeline, X.columns),
    ]
)

In [71]:
xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(**xgb_best_params)),
])

In [72]:
scores,oof = cross_validation(xgb, X, y, le)

Fold 1


/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:52:05] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Fold 2


/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:52:51] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Fold 3


...

Fold 4


/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:54:11] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Fold 5


/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:54:53] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Score: 0.8315584022823552


In [74]:
for fold in oof:
    fold['true'] = fold['true'].replace(le.classes_)

save_oof(oof)

In [76]:
X_test = test[initial_features]

predict_test(xgb, X, y, X_test, le.classes_)

/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:15:51] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


,pred_Dropout,pred_Enrolled,pred_Graduate
0,0.995347,0.002031,0.002622
1,0.004109,0.013508,0.982382
2,0.051927,0.221811,0.726261
3,0.274529,0.341359,0.384113
4,0.279766,0.677566,0.042669
...,...,...,...
51007,0.909216,0.037486,0.053298
51008,0.984859,0.014725,0.000416
51009,0.971530,0.012638,0.015832
51010,0.883119,0.100866,0.016015
